In [2]:
!pip install pyyaml

In [3]:
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

torch:  2.5 ; cuda:  cu121


In [4]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-c38gychu
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-c38gychu
  Resolved https://github.com/facebookresearch/detectron2.git to commit b1c43ffbc995426a9a6b5c667730091a384e0fa4
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.4 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp3

In [5]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os
import json
import cv2
import random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("fold_1_train", {}, "/content/drive/MyDrive/Cv-Project/Splits/fold_1_train.json", "/content/drive/MyDrive/Cv-Project/Data")
register_coco_instances("fold_1_val", {}, "/content/drive/MyDrive/Cv-Project/Splits/fold_1_val.json", "/content/drive/MyDrive/Cv-Project/Data")

register_coco_instances("fold_2_train", {}, "/content/drive/MyDrive/Cv-Project/Splits/fold_2_train.json", "/content/drive/MyDrive/Cv-Project/Data")
register_coco_instances("fold_2_val", {}, "/content/drive/MyDrive/Cv-Project/Splits/fold_2_val.json", "/content/drive/MyDrive/Cv-Project/Data")

register_coco_instances("fold_3_train", {}, "/content/drive/MyDrive/Cv-Project/Splits/fold_3_train.json", "/content/drive/MyDrive/Cv-Project/Data")
register_coco_instances("fold_3_val", {}, "/content/drive/MyDrive/Cv-Project/Splits/fold_3_val.json", "/content/drive/MyDrive/Cv-Project/Data")

register_coco_instances("fold_4_train", {}, "/content/drive/MyDrive/Cv-Project/Splits/fold_4_train.json", "/content/drive/MyDrive/Cv-Project/Data")
register_coco_instances("fold_4_val", {}, "/content/drive/MyDrive/Cv-Project/Splits/fold_4_val.json", "/content/drive/MyDrive/Cv-Project/Data")

register_coco_instances("fold_5_train", {}, "/content/drive/MyDrive/Cv-Project/Splits/fold_5_train.json", "/content/drive/MyDrive/Cv-Project/Data")
register_coco_instances("fold_5_val", {}, "/content/drive/MyDrive/Cv-Project/Splits/fold_5_val.json", "/content/drive/MyDrive/Cv-Project/Data")



#### Train Section

In [9]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("fold_1_train",)
cfg.DATASETS.TEST = ("fold_1_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.MAX_ITER = 50
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.OUTPUT_DIR = ".drive/MyDrive/CV-Project/output"

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[12/21 10:42:30 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[12/21 10:42:31 d2.engine.train_loop]: Starting training from iteration 0
[12/21 10:42:40 d2.utils.events]:  eta: 0:00:13  iter: 19  total_loss: 1.572  loss_cls: 0.9786  loss_box_reg: 0.3125  loss_rpn_cls: 0.09747  loss_rpn_loc: 0.02343    time: 0.4517  last_time: 0.5300  data_time: 0.0197  last_data_time: 0.0059   lr: 0.00095155  max_mem: 3117M
[12/21 10:42:49 d2.utils.events]:  eta: 0:00:04  iter: 39  total_loss: 1.198  loss_cls: 0.5099  loss_box_reg: 0.6339  loss_rpn_cls: 0.04467  loss_rpn_loc: 0.01429    time: 0.4430  last_time: 0.4266  data_time: 0.0073  last_data_time: 0.0018   lr: 0.0019506  max_mem: 3117M
[12/21 10:42:55 d2.utils.events]:  eta: 0:00:00  iter: 49  total_loss: 1.379  loss_cls: 0.4561  loss_box_reg: 0.7931  loss_rpn_cls: 0.04467  loss_rpn_loc: 0.01556    time: 0.4469  last_time: 0.4623  data_time: 0.0106  last_data_time: 0.0268   lr: 0.0024501  max_mem: 3118M
[12/21 10:42:55 d2.engine.hooks]: Overall training speed: 48 iterations in 0:00:21 (0.4469 s / it)
[12/21 

In [10]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [12]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.DATASETS.TEST = ("fold_1_val",)
predictor = DefaultPredictor(cfg)

[12/21 10:58:31 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...


/usr/local/lib/python3.10/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

In [13]:
evaluator = COCOEvaluator("fold_1_val", output_dir= "./output")
val_loader = build_detection_test_loader(cfg, "fold_1_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

[12/21 10:58:36 d2.data.datasets.coco]: Loaded 371 images in COCO format from /content/drive/MyDrive/Cv-Project/Splits/fold_1_val.json
[12/21 10:58:36 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/21 10:58:36 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[12/21 10:58:36 d2.data.common]: Serializing 371 elements to byte tensors and concatenating them all ...
[12/21 10:58:36 d2.data.common]: Serialized dataset takes 0.16 MiB
[12/21 10:58:36 d2.evaluation.evaluator]: Start inference on 371 batches
[12/21 10:58:38 d2.evaluation.evaluator]: Inference done 11/371. Dataloading: 0.0753 s/iter. Inference: 0.1016 s/iter. Eval: 0.0001 s/iter. Total: 0.1770 s/iter. ETA=0:01:03
[12/21 10:58:43 d2.evaluation.evaluator]: Inference done 33/371. Dataloading: 0.1171 s/iter. Inference: 0.1034 s/iter. Eval: 0.0002 s/iter. Total: 0.

In [14]:
f = open('config.yml', 'w')
f.write(cfg.dump())
f.close()

In [9]:
import os
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine.defaults import DefaultPredictor

In [10]:
# Paths
splits_path = "/content/drive/MyDrive/Cv-Project/Splits/"
data_path = "/content/drive/MyDrive/Cv-Project/Data"
output_path = "/content/drive/MyDrive/CV-Project/output"

In [11]:
def process_fold(fold):
    train_name = f"fold_{fold}_train"
    val_name = f"fold_{fold}_val"
    register_coco_instances(train_name, {}, f"{splits_path}fold_{fold}_train.json", data_path)
    register_coco_instances(val_name, {}, f"{splits_path}fold_{fold}_val.json", data_path)

    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
    cfg.DATASETS.TRAIN = (train_name,)
    cfg.DATASETS.TEST = (val_name,)
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = 0.0025
    cfg.SOLVER.MAX_ITER = 10000
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
    cfg.OUTPUT_DIR = f"{output_path}/fold_{fold}"
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()

    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    predictor = DefaultPredictor(cfg)

    evaluator = COCOEvaluator(val_name, output_dir=cfg.OUTPUT_DIR)
    val_loader = build_detection_test_loader(cfg, val_name)
    eval_results = inference_on_dataset(predictor.model, val_loader, evaluator)

    print(f"Fold {fold} results:")
    print(eval_results)
    with open(os.path.join(cfg.OUTPUT_DIR, "eval_results.txt"), "w") as f:
        f.write(str(eval_results))

    with open(os.path.join(cfg.OUTPUT_DIR, "config.yml"), "w") as f:
        f.write(cfg.dump())

    return eval_results

In [25]:
fold_1_results = process_fold(1)

[12/21 11:59:43 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[12/21 11:59:44 d2.engine.train_loop]: Starting training from iteration 0
[12/21 11:59:53 d2.utils.events]:  eta: 1:17:49  iter: 19  total_loss: 1.786  loss_cls: 1.329  loss_box_reg: 0.2154  loss_rpn_cls: 0.1455  loss_rpn_loc: 0.02583    time: 0.4566  last_time: 0.4976  data_time: 0.0222  last_data_time: 0.0186   lr: 4.9952e-05  max_mem: 3919M
[12/21 12:00:01 d2.utils.events]:  eta: 1:13:23  iter: 39  total_loss: 1.282  loss_cls: 0.8527  loss_box_reg: 0.299  loss_rpn_cls: 0.07792  loss_rpn_loc: 0.02618    time: 0.4368  last_time: 0.3598  data_time: 0.0096  last_data_time: 0.0036   lr: 9.9902e-05  max_mem: 3919M
[12/21 12:00:11 d2.utils.events]:  eta: 1:14:29  iter: 59  total_loss: 1.289  loss_cls: 0.6149  loss_box_reg: 0.5522  loss_rpn_cls: 0.06251  loss_rpn_loc: 0.01992    time: 0.4427  last_time: 0.3257  data_time: 0.0102  last_data_time: 0.0189   lr: 0.00014985  max_mem: 3919M
[12/21 12:00:19 d2.utils.events]:  eta: 1:14:20  iter: 79  total_loss: 1.052  loss_cls: 0.4614  loss_box_re

/usr/local/lib/python3.10/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

[12/21 13:13:07 d2.data.datasets.coco]: Loaded 371 images in COCO format from /content/drive/MyDrive/Cv-Project/Splits/fold_1_val.json
[12/21 13:13:07 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/21 13:13:07 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[12/21 13:13:07 d2.data.common]: Serializing 371 elements to byte tensors and concatenating them all ...
[12/21 13:13:07 d2.data.common]: Serialized dataset takes 0.16 MiB
[12/21 13:13:07 d2.evaluation.evaluator]: Start inference on 371 batches
[12/21 13:13:09 d2.evaluation.evaluator]: Inference done 11/371. Dataloading: 0.0110 s/iter. Inference: 0.1148 s/iter. Eval: 0.0003 s/iter. Total: 0.1262 s/iter. ETA=0:00:45
[12/21 13:13:14 d2.evaluation.evaluator]: Inference done 56/371. Dataloading: 0.0033 s/iter. Inference: 0.1100 s/iter. Eval: 0.0004 s/iter. Total: 0.

In [26]:
fold_2_results = process_fold(2)

[12/21 13:13:51 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[12/21 13:13:52 d2.engine.train_loop]: Starting training from iteration 0
[12/21 13:14:01 d2.utils.events]:  eta: 1:14:32  iter: 19  total_loss: 1.741  loss_cls: 1.401  loss_box_reg: 0.2387  loss_rpn_cls: 0.1548  loss_rpn_loc: 0.03497    time: 0.4539  last_time: 0.4398  data_time: 0.0300  last_data_time: 0.0074   lr: 4.9952e-05  max_mem: 3919M
[12/21 13:14:10 d2.utils.events]:  eta: 1:13:00  iter: 39  total_loss: 1.422  loss_cls: 0.8272  loss_box_reg: 0.3817  loss_rpn_cls: 0.08651  loss_rpn_loc: 0.03359    time: 0.4425  last_time: 0.5321  data_time: 0.0097  last_data_time: 0.0249   lr: 9.9902e-05  max_mem: 3919M
[12/21 13:14:19 d2.utils.events]:  eta: 1:14:11  iter: 59  total_loss: 1.004  loss_cls: 0.5441  loss_box_reg: 0.4143  loss_rpn_cls: 0.05711  loss_rpn_loc: 0.02715    time: 0.4494  last_time: 0.5291  data_time: 0.0112  last_data_time: 0.0052   lr: 0.00014985  max_mem: 3919M
[12/21 13:14:28 d2.utils.events]:  eta: 1:14:27  iter: 79  total_loss: 1.014  loss_cls: 0.4932  loss_box_r

/usr/local/lib/python3.10/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

[12/21 14:27:07 d2.data.datasets.coco]: Loaded 370 images in COCO format from /content/drive/MyDrive/Cv-Project/Splits/fold_2_val.json
[12/21 14:27:07 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/21 14:27:07 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[12/21 14:27:07 d2.data.common]: Serializing 370 elements to byte tensors and concatenating them all ...
[12/21 14:27:07 d2.data.common]: Serialized dataset takes 0.17 MiB
[12/21 14:27:07 d2.evaluation.evaluator]: Start inference on 370 batches
[12/21 14:27:09 d2.evaluation.evaluator]: Inference done 11/370. Dataloading: 0.0124 s/iter. Inference: 0.1144 s/iter. Eval: 0.0003 s/iter. Total: 0.1270 s/iter. ETA=0:00:45
[12/21 14:27:14 d2.evaluation.evaluator]: Inference done 51/370. Dataloading: 0.0063 s/iter. Inference: 0.1187 s/iter. Eval: 0.0004 s/iter. Total: 0.

In [29]:
fold_3_results = process_fold(3)

[12/21 16:15:03 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[12/21 16:15:03 d2.engine.train_loop]: Starting training from iteration 0
[12/21 16:15:12 d2.utils.events]:  eta: 1:17:38  iter: 19  total_loss: 1.677  loss_cls: 1.253  loss_box_reg: 0.2046  loss_rpn_cls: 0.1282  loss_rpn_loc: 0.02391    time: 0.4630  last_time: 0.4953  data_time: 0.0297  last_data_time: 0.0061   lr: 4.9952e-05  max_mem: 5476M
[12/21 16:15:21 d2.utils.events]:  eta: 1:16:31  iter: 39  total_loss: 1.398  loss_cls: 0.7772  loss_box_reg: 0.3547  loss_rpn_cls: 0.1049  loss_rpn_loc: 0.02863    time: 0.4555  last_time: 0.5095  data_time: 0.0098  last_data_time: 0.0190   lr: 9.9902e-05  max_mem: 5476M
[12/21 16:15:31 d2.utils.events]:  eta: 1:16:51  iter: 59  total_loss: 1.025  loss_cls: 0.506  loss_box_reg: 0.4301  loss_rpn_cls: 0.08775  loss_rpn_loc: 0.02478    time: 0.4570  last_time: 0.5404  data_time: 0.0093  last_data_time: 0.0082   lr: 0.00014985  max_mem: 5565M
[12/21 16:15:39 d2.utils.events]:  eta: 1:15:53  iter: 79  total_loss: 1.118  loss_cls: 0.5106  loss_box_reg

/usr/local/lib/python3.10/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

[12/21 17:28:23 d2.data.datasets.coco]: Loaded 370 images in COCO format from /content/drive/MyDrive/Cv-Project/Splits/fold_3_val.json
[12/21 17:28:23 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/21 17:28:23 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[12/21 17:28:23 d2.data.common]: Serializing 370 elements to byte tensors and concatenating them all ...
[12/21 17:28:23 d2.data.common]: Serialized dataset takes 0.16 MiB
[12/21 17:28:23 d2.evaluation.evaluator]: Start inference on 370 batches
[12/21 17:28:25 d2.evaluation.evaluator]: Inference done 11/370. Dataloading: 0.0044 s/iter. Inference: 0.1144 s/iter. Eval: 0.0005 s/iter. Total: 0.1193 s/iter. ETA=0:00:42
[12/21 17:28:30 d2.evaluation.evaluator]: Inference done 53/370. Dataloading: 0.0042 s/iter. Inference: 0.1147 s/iter. Eval: 0.0005 s/iter. Total: 0.

In [12]:
fold_4_results = process_fold(4)

[12/21 19:13:21 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_280758.pkl: 167MB [00:00, 196MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[12/21 19:13:24 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/21 19:13:52 d2.utils.events]:  eta: 2:18:18  iter: 19  total_loss: 1.907  loss_cls: 1.399  loss_box_reg: 0.2801  loss_rpn_cls: 0.127  loss_rpn_loc: 0.01691    time: 0.8480  last_time: 0.9338  data_time: 0.8427  last_data_time: 0.5906   lr: 4.9952e-05  max_mem: 2424M
[12/21 19:14:12 d2.utils.events]:  eta: 2:08:15  iter: 39  total_loss: 1.293  loss_cls: 0.7832  loss_box_reg: 0.3698  loss_rpn_cls: 0.0862  loss_rpn_loc: 0.01585    time: 0.8039  last_time: 0.4608  data_time: 0.3753  last_data_time: 0.0042   lr: 9.9902e-05  max_mem: 2637M
[12/21 19:14:29 d2.utils.events]:  eta: 2:13:35  iter: 59  total_loss: 0.9693  loss_cls: 0.4899  loss_box_reg: 0.3276  loss_rpn_cls: 0.06938  loss_rpn_loc: 0.02482    time: 0.8108  last_time: 0.8644  data_time: 0.4369  last_data_time: 0.4316   lr: 0.00014985  max_mem: 2637M
[12/21 19:14:45 d2.utils.events]:  eta: 2:15:22  iter: 79  total_loss: 1.134  loss_cls: 0.5049  loss_box_reg: 0.5774  loss_rpn_cls: 0.03858  loss_rpn_loc: 0.02192    time: 0.8051  l

/usr/local/lib/python3.10/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

[12/21 20:29:05 d2.data.datasets.coco]: Loaded 370 images in COCO format from /content/drive/MyDrive/Cv-Project/Splits/fold_4_val.json
[12/21 20:29:05 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/21 20:29:05 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[12/21 20:29:05 d2.data.common]: Serializing 370 elements to byte tensors and concatenating them all ...
[12/21 20:29:05 d2.data.common]: Serialized dataset takes 0.15 MiB
[12/21 20:29:05 d2.evaluation.evaluator]: Start inference on 370 batches
[12/21 20:29:10 d2.evaluation.evaluator]: Inference done 11/370. Dataloading: 0.3120 s/iter. Inference: 0.0954 s/iter. Eval: 0.0003 s/iter. Total: 0.4077 s/iter. ETA=0:02:26
[12/21 20:29:16 d2.evaluation.evaluator]: Inference done 23/370. Dataloading: 0.3261 s/iter. Inference: 0.1037 s/iter. Eval: 0.0004 s/iter. Total: 0.

In [13]:
fold_5_results = process_fold(5)

[12/21 20:42:02 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[12/21 20:42:03 d2.engine.train_loop]: Starting training from iteration 0
[12/21 20:42:11 d2.utils.events]:  eta: 1:06:04  iter: 19  total_loss: 1.535  loss_cls: 1.124  loss_box_reg: 0.2361  loss_rpn_cls: 0.1296  loss_rpn_loc: 0.02431    time: 0.3812  last_time: 0.4100  data_time: 0.0192  last_data_time: 0.0106   lr: 4.9952e-05  max_mem: 2671M
[12/21 20:42:19 d2.utils.events]:  eta: 1:07:05  iter: 39  total_loss: 1.153  loss_cls: 0.6814  loss_box_reg: 0.2728  loss_rpn_cls: 0.08878  loss_rpn_loc: 0.02548    time: 0.4021  last_time: 0.4573  data_time: 0.0144  last_data_time: 0.0065   lr: 9.9902e-05  max_mem: 2671M
[12/21 20:42:27 d2.utils.events]:  eta: 1:06:57  iter: 59  total_loss: 1.087  loss_cls: 0.5016  loss_box_reg: 0.3815  loss_rpn_cls: 0.08311  loss_rpn_loc: 0.02838    time: 0.4040  last_time: 0.3916  data_time: 0.0078  last_data_time: 0.0064   lr: 0.00014985  max_mem: 2671M
[12/21 20:42:36 d2.utils.events]:  eta: 1:07:11  iter: 79  total_loss: 1.27  loss_cls: 0.538  loss_box_reg

/usr/local/lib/python3.10/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

[12/21 21:52:48 d2.data.datasets.coco]: Loaded 370 images in COCO format from /content/drive/MyDrive/Cv-Project/Splits/fold_5_val.json
[12/21 21:52:48 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/21 21:52:48 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[12/21 21:52:48 d2.data.common]: Serializing 370 elements to byte tensors and concatenating them all ...
[12/21 21:52:48 d2.data.common]: Serialized dataset takes 0.16 MiB
[12/21 21:52:48 d2.evaluation.evaluator]: Start inference on 370 batches
[12/21 21:52:49 d2.evaluation.evaluator]: Inference done 11/370. Dataloading: 0.0016 s/iter. Inference: 0.1068 s/iter. Eval: 0.0003 s/iter. Total: 0.1088 s/iter. ETA=0:00:39
[12/21 21:52:54 d2.evaluation.evaluator]: Inference done 46/370. Dataloading: 0.0077 s/iter. Inference: 0.1310 s/iter. Eval: 0.0005 s/iter. Total: 0.